In [1]:
import tensorflow as tf

In [2]:
import os

import numpy as np
import pandas as pd

In [3]:
os.environ["CUDA_VISIBLE_DIVICES"] = "0"

In [4]:
data_df = pd.read_csv("../data/baidu_sentence.csv")

In [5]:
data_df.head()

,labels,content,subject
0,高中 生物 分子与细胞 组成细胞的化学元素 组成细胞的化合物,菠菜 土壤 中 吸收 氮 元素 用来 合成 淀粉 纤维素 葡萄糖 核酸 蛋白质 麦芽糖 脂肪酸,生物
1,高中 生物 稳态与环境 神经调节和体液调节的比较,下列 生物体 内 信息 传递 叙述 正确 下丘脑 分泌 促 甲状腺 激素 释放 激素 作用 ...,生物
2,高中 生物 生物技术实践 生物工程技术,自然 菌样 筛选 理想 生产 菌种 步骤 采集 菌样 富集 培养 纯种 分离 性能 测定 不...,生物
3,高中 生物 生物技术实践 生物技术在其他方面的应用 器官移植 复等位基因 胚胎移植 基因工程...,目前 精子 载体 法 逐渐 成为 具有 诱惑力 制备 转基因 动物 方法 方法 精子 外源 ...,生物
4,高中 地理 宇宙中的地球 地球运动的地理意义,某人 想 普通 飞机 一年 中 连续 两次 生日 认为 应 穿越 赤道 两级 本初子午线 国...,地理


In [25]:
from collections import Counter
import json

def gen_vocabulary(contents, labels):
    # 拉平词语
    all_words = [word for content in contents for word in content]
    print("all words:", len(all_words))
    # 去除低频词
    word_counts = Counter(all_words)
    sorted_words = sorted(word_counts.items(), key=lambda x:x[1], reverse=True)
    fw_words = [word[0] for word in sorted_words if word[1] > 5]
    print("after clear low frequence:", len(fw_words))
    
    vocab, wordEmbedding = get_word_embedding(fw_words)
    
    word2idx = dict(zip(vocab, list(range(len(vocab)))))
    label2idx = dict(zip(list(set(labels)), range(len(list(set(labels))))))
    
    # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
    with open("../data/wordJson/word2idx.json", "w", encoding="utf-8") as f:
        json.dump(word2idx, f)
        
    with open("../data/wordJson/label2idx.json", "w", encoding="utf-8") as f:
            json.dump(label2idx, f)
        
    return word2idx, label2idx, vocab, wordEmbedding

In [26]:
import gensim
def get_word_embedding(words):
    word2vec = gensim.models.KeyedVectors.load_word2vec_format("../data/word2vec.bin", binary=True)
    vocab = []
    wordEmbedding = []
    
    vocab.append("PAD")
    vocab.append("UNK")
    wordEmbedding.append(np.zeros(200))
    wordEmbedding.append(np.random.randn(200))
    
    for word in words:
        try:
            vector = word2vec.wv[word]
            vocab.append(word)
            wordEmbedding.append(vector)
        except:
            print(word + "not exist in word2vec.")
    
    return vocab, np.array(wordEmbedding)

In [27]:
contents = [content.split() for content in data_df["content"].tolist()]

In [28]:
labels = data_df["subject"].tolist()

In [29]:
word2idx, label2idx, vocab, wordEmbedding = gen_vocabulary(contents, labels)

all words: 1019403
after clear low frequence: 12602


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [31]:
wordEmbedding.shape

(12604, 200)

In [32]:
len(vocab)

12604

## 标签数值化

In [33]:
def labelToIndex(labels, label2idx):
    labelIds = [label2idx[label] for label in labels]
    
    return labelIds

def contentToIndex(contents, word2idx):
    contentIds = [[word2idx.get(word, word2idx["UNK"]) for word in content] for content in contents]
    
    return contentIds

In [34]:
labelsIds = labelToIndex(labels, label2idx)

In [36]:
contentIds = contentToIndex(contents, word2idx)

In [41]:
contentIds[:1]

[[3396,
  402,
  2,
  495,
  2481,
  605,
  1768,
  107,
  1002,
  682,
  359,
  470,
  55,
  2290,
  3863]]

In [55]:
def gen_train_eval_data(x, y, word2idx, max_len, rate=0.8):
    contents = []
    
    for content in x:
        if len(content) >= max_len:
            contents.append(content[:max_len])
        else:
            contents.append(content + [word2idx["PAD"]] * (max_len - len(content)))
            
    split_idx = int(len(x) * rate)
    
    train_data = np.asarray(contents[:split_idx], dtype="int64")
    train_label = np.array(y[:split_idx], dtype="float32")
    
    eval_data = np.asarray(contents[split_idx:], dtype="int64")
    eval_label = np.array(y[split_idx:], dtype="float32")
    
    return train_data, train_label, eval_data, eval_label

In [56]:
def get_max_len(data):
    max_lens = data.apply(lambda x: x.count(" "))
    
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [57]:
train_x_max_len = get_max_len(data_df["content"])

In [58]:
train_x_max_len

108

In [59]:
train_data, train_label, eval_data, eval_label = gen_train_eval_data(contentIds, labelsIds, word2idx, get_max_len(data_df["content"]), 0.8)

In [60]:
train_data.shape

(18060, 108)

In [62]:
eval_data.shape

(4516, 108)

# 模型

## 1. 生成batch数据集

In [63]:
def next_batch(x, y, batch_size):
    perm = np.arange(len(x))
    np.random.shuffle(perm)
    
    x = x[perm]
    y = y[perm]
    
    num_batches = len(x) // batch_size
    
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        
        batchX = np.array(x[start:end], dtype="int64")
        batchY = np.array(y[start:end], dtype="int64")
        
    return batchX, batchY